In [2]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
from scipy import signal
import random

import h5py
from IPython.utils import io

In [3]:
from joblib import Parallel, delayed
from tqdm import tqdm

In [4]:
%matplotlib widget

In [5]:
import sys
sys.path.append('../../../../nadavp/new_trap/Tools/')
sys.path.append('../../../../nadavp/new_trap/Tools/StatFramework/')
import BeadDataFile
from discharge_tools import *

sys.path.append('/home/analysis_user/New_trap_code/SensitivityFramework/')
from signal_model_utilities import *

In [6]:
%reload_ext autoreload
%autoreload 2

In [7]:
from likelihood_calculator import likelihood_analyser
from likelihood_calculator import GravityFramework
from likelihood_calculator import auxiliary_functions as aux
gfw = GravityFramework.GravityFramework()

## Discahrge

In [16]:
bdf_discharge = load_dir(dirname='/data/new_trap/20200924/Bead1/Discharge/Recharge_20200925/', start_file=0)

70  files in folder
70  files loaded


In [11]:
bb = bdf_discharge[0]
bb.electrode_channels

array([1, 2], dtype=uint16)

In [27]:
dis_z = [np.std(bb.response_at_freq2('z', drive_freq=71)) for bb in bdf_discharge]

In [74]:
ch = [bb.electrode_channels[0] for bb in bdf_discharge]

In [29]:
_,ax = plt.subplots(figsize=(9.1,4))
ax.scatter(range(len(dis_z)), dis_z/dis_z[0], label='std')
ax.scatter(range(len(dis_z)), amps/amps[0], label='fit')
ax.legend()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [75]:
bdf_discharge_z = [bb for bb,ch_ in zip(bdf_discharge, ch)  if ch_==1 ]
bdf_discharge_x = [bb for bb,ch_ in zip(bdf_discharge, ch)  if ch_==4 ]

In [76]:
index_z = [bb for bb,ch_ in zip(range(len(bdf_discharge)), ch)  if ch_==1 ]
index_x = [bb for bb,ch_ in zip(range(len(bdf_discharge)), ch)  if ch_==4 ]

In [71]:
m1 = gfw.build_z_response(bdf_discharge, drive_freq=71, charges=1, bandwidth=2, decimate=10, include_sigma=True, bimodal=False)

***************************************************
Z2-amplitude:  4.34e+02
reduced chi2:  9.318084811908948
***************************************************
Z2-amplitude:  4.09e+02
reduced chi2:  9.574709644861358
***************************************************
Z2-amplitude:  3.72e+02
reduced chi2:  9.824914975818828
***************************************************
Z2-amplitude:  4.44e+02
reduced chi2:  9.421113095866765
***************************************************
Z2-amplitude:  4.25e+02
reduced chi2:  9.44492630131021
***************************************************
Z2-amplitude:  3.36e+02
reduced chi2:  9.305279518412256
***************************************************
Z2-amplitude:  3.18e+02
reduced chi2:  9.293684699273042
***************************************************
Z2-amplitude:  3.31e+02
reduced chi2:  9.502907961635959
***************************************************
Z2-amplitude:  3.22e+02
reduced chi2:  9.184956380784522
********************

In [15]:
m2 = gfw.build_x_response(bdf_discharge_x, drive_freq=71, charges=1, bandwidth=1)

***************************************************
X2-amplitude:  1.22e+01
reduced chi2:  1.5077204319970472
***************************************************
X2-amplitude:  1.22e+01
reduced chi2:  1.6561307426139402
***************************************************
X2-amplitude:  1.23e+01
reduced chi2:  1.2313121190426974
***************************************************
X2-amplitude:  1.22e+01
reduced chi2:  1.16464891136188
***************************************************
X2-amplitude:  1.21e+01
reduced chi2:  1.2168587862400966
***************************************************
X2-amplitude:  1.22e+01
reduced chi2:  1.2696742562620946
***************************************************
X2-amplitude:  1.23e+01
reduced chi2:  1.458111243801847
***************************************************
X2-amplitude:  1.23e+01
reduced chi2:  1.2708153177285613
***************************************************
X2-amplitude:  1.22e+01
reduced chi2:  1.4039743199327412
*************

In [77]:
dis_x = [np.std(bb.response_at_freq('x', drive_freq=71)) for bb in bdf_discharge_x]
dis_z = [np.std(bb.response_at_freq2('z', drive_freq=71)) for bb in bdf_discharge]

In [72]:
amps = np.array([m_.values[0] for m_ in m1])
phases = np.array([m_.values[3] for m_ in m1])
_, ax = plt.subplots(1,2,figsize=(9.1,4))
ax[0].scatter(range(len(amps)), amps)
ax[1].scatter(range(len(amps)), phases)
np.mean(phases[20:50])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

71.74075198338863

In [80]:
_,ax = plt.subplots(figsize=(9.5,4))
scale = 81
scale2 = 50
scaleX = -0.8
scaleX2 = 9200

# Perrinf
ax.scatter(np.arange(len(amps)), amps/scale, label='fit fixed-phase')
# ax.scatter(range(len(dis_z)), np.array(dis_z)/scale2, label='Z response')
# ax.scatter(range(len(dis_z)), np.array(dis_z)/scale2, label='Z response')
# ax.scatter(index_x, ampsx/scaleX, label='X response')
ax.scatter(index_x, np.array(dis_x)/scaleX2, label='X-pos fit fixed-phases')
ax.scatter(np.arange(len(amps)), ch, label='channel')

ax.legend()
ax.set(xlabel='dataset', ylabel='Charges', ylim=(None,None), xlim=(None,None))
ax.grid(axis='y')
ax.minorticks_on()
ax.grid(which='major', linestyle='-', linewidth='0.5', color='red')
ax.grid(which='minor', linestyle=':', linewidth='0.5', color='black')
ax.legend()
from matplotlib.ticker import MaxNLocator
ax.yaxis.set_major_locator(MaxNLocator(integer=True))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## TF

In [15]:
# 7.6um German bead
fname = r'/home/data_transfer/data_from_windows/20200921/Bead1/TransFunc/TransFuncX2_1Hz_300s.h5'
tf_x = BeadDataFile.BeadDataFile(fname=fname)
fname = r'/home/data_transfer/data_from_windows/20200921/Bead1/TransFunc/TransFuncY1_1Hz_100s.h5'
tf_y = BeadDataFile.BeadDataFile(fname=fname)
fname = r'/home/data_transfer/data_from_windows/20200921/Bead1/TransFunc/TransFuncZ2_1Hz_300s.h5'
tf_z = BeadDataFile.BeadDataFile(fname=fname)
tf_list = [tf_x, tf_y, tf_z]

No height data
No cantilever data
Error loading spinning data
No laser power data
No height data
No cantilever data
Error loading spinning data
No laser power data
No height data
No cantilever data
Error loading spinning data
No laser power data


In [8]:
_,ax = plt.subplots(1,3, figsize=(9,3))
bb = tf_z
ax[0].semilogy(*bb.psd2('x'))
ax[1].semilogy(*bb.psd2('y'))
ax[2].semilogy(*bb.psd2('z'))
[a.set(xlim=(0,100)) for a in ax]

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

[[(0.0, 100.0)], [(0.0, 100.0)], [(0.0, 100.0)]]

In [50]:
names = ['Y2', 'Y3', 'Y4', 'Y5', 'Y6', 'Y7', 'Y8', 'Y9']
names = ['Y7', 'Y8', 'Y9', 'Y11']
fnames = [r'/home/data_transfer/data_from_windows/20200921/Bead1/TransFunc/TransFunc'+y_+'_7Hz_10s.h5' for y_ in names]
bbs = [BeadDataFile.BeadDataFile(fname_) for fname_ in fnames]

No height data
No cantilever data
Error loading spinning data
No laser power data
No height data
No cantilever data
Error loading spinning data
No laser power data
No height data
No cantilever data
Error loading spinning data
No laser power data
No height data
No cantilever data
Error loading spinning data
No laser power data


In [56]:
_,ax = plt.subplots()
[ax.semilogy(*bb.psd2('y'), label=name_) for bb,name_ in zip(bbs,names)]
ax.legend()
ax.set(xlim=(100,1000))
tmp_freq, tmp_psd = bbs[-1].psd2('x')
np.sqrt(tmp_psd[tmp_freq==71]/tmp_psd[tmp_freq==50])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

array([2.06282349])

## Nois floor

In [7]:
bdf_z = bdf_discharge[40:45]
bdf_x = bdf_discharge[30:35]

freq = bdf_z[0].psd2('z')[0]
z_scale1 = bdf_z[0].psd2('z')[1][freq==71]
for bb in bdf_z:
    z_scale1 += bb.psd2('z')[1][freq==71]
z_scale1 /= len(bdf_z)

x_scale1 = bdf_x[0].psd2('x')[1][freq==71]
for bb in bdf_x:
    x_scale1 += bb.psd2('x')[1][freq==71]
x_scale1 /= len(bdf_x)

charges = 3
force = charges * 1.6e-19 * 20 / 8.6e-3 * 0.63 * np.sqrt(10) # in Newtons/sqrt(Hz)
z_scale1 = force/np.sqrt(z_scale1)
x_scale1 = force/np.sqrt(x_scale1)

In [8]:
del bdf_discharge

In [91]:
## 100 files of shaking data for nose floor
num_files = 100
data_bdfs = load_dir(dirname='/data/new_trap/20200924/Bead1/Shaking/Shaking345/',
                         file_prefix='SpinShaking2_', start_file=0, max_file=num_files)
freq = data_bdfs[0].psd2('x')[0]
fftx1 = data_bdfs[0].psd2('x')[1]
ffty1 = data_bdfs[0].psd2('y')[1]
fftz1 = data_bdfs[0].psd2('z')[1]
for i in range(1,num_files):
    fftx1 += data_bdfs[i].psd2('x')[1]
    fftz1 += data_bdfs[i].psd2('z')[1]
    ffty1 += data_bdfs[i].psd2('y')[1]
fftx1/=num_files
ffty1/=num_files
fftz1/=num_files

157  files in folder
100  files loaded


In [92]:
_,ax = plt.subplots(1,2,figsize=(9.1,4), sharex=True)
# tf_freq = np.interp(freq, gfw.tf_freq, gfw.tf_ffts[2])
ax[0].semilogy(freq, np.sqrt(fftx)*x_scale1, label='Spin')
ax[0].semilogy(freq, np.sqrt(fftx1)*x_scale1, label='Ringdown', alpha=0.6)
ax[1].semilogy(freq, np.sqrt(fftz)*z_scale1, label='Spin')
ax[1].semilogy(freq, np.sqrt(fftz1)*z_scale1, label='Ringdown', alpha=0.6)
[a.legend() for a in ax]
ax[0].set(xlim=(1,50), ylim=(9e-18,1e-14), xlabel='Frequency [Hz.]', ylabel=r'ASD [N/$\sqrt{Hz}$]', title='X force')
_ = ax[1].set(xlim=(1,50), ylim=(9e-18,1e-14), xlabel='Frequency [Hz.]', title=r'Z force')
# print(np.sqrt(fftx1[freq==18])*x_scale/np.sqrt(10), np.sqrt(fftz1[freq==18])*z_scale/np.sqrt(10))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [63]:
## 100 files of shaking data for nose floor
num_files = 100
data_bdfs = load_dir(dirname='/data/new_trap/20200924/Bead1/Shaking/Shaking340/',
                         file_prefix='NoSpinNoShaking0_', start_file=500, max_file=num_files)
bb = data_bdfs[0]
freq = np.fft.rfftfreq(len(bb.spin_data[0]), d=1./5000/40)
fft11 = np.abs(np.fft.rfft(bb.spin_data[0]))**2
for i in range(1,num_files):
    fft11 += np.abs(np.fft.rfft(data_bdfs[i].spin_data[0]))**2
fft11/=num_files

_,ax = plt.subplots()
ax.semilogy(freq, fft)
ax.semilogy(freq, fft11)
ax.set(xlim=(1,5000))

2000  files in folder
100  files loaded


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

[(1.0, 5000.0)]

In [87]:
def func1_height(i):
    print(i,'  ******************')
    fname = r'/data/new_trap/20200924/Bead1/Shaking/Shaking345/SpinShaking0_'+str(i)+'.h5'
    bb = BeadDataFile.BeadDataFile(fname=fname)
    return np.mean(bb.bead_height)

In [88]:
func1_height(101)

101   ******************


180.12130227424302

In [77]:
from joblib import Parallel, delayed
from tqdm import tqdm
test1_zfb = Parallel(n_jobs=20)(delayed(func1_height)(i) for i in tqdm(range(2500)))

100%|██████████| 2500/2500 [00:09<00:00, 257.42it/s]


In [79]:
_,ax = plt.subplots(figsize=(9.2,4))
ax.scatter(range(2500), test1_zfb)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [17]:
i = 1400
fname = r'/data/new_trap/20200921/Bead1/Discharge/HeightControl2/Discharge_'+str(i)+'.h5'
bb = BeadDataFile.BeadDataFile(fname=fname)

In [18]:
f = h5py.File(fname,'r')
pos_data = np.array(f['pos_data'])
quad_data = np.array(f['quad_data'])

In [20]:
sync = pos_data.reshape(-1,11).T[5]

In [21]:
_,ax = plt.subplots(figsize=(9.5,4))
ax.plot(range(500), sync[:500]/100)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [62]:
i_start = np.nonzero(sync[:500])[0][0]
zfb = bb.feedback[2][i_start-1:]
x_sync = bb.xyz[0][i_start-1:]

In [65]:
_,ax = plt.subplots(figsize=(9.2,4))
ax.plot(range(40000), zfb[100000:140000]/np.mean(zfb[:40000]), label='z_fb')
ax.plot(range(40000), bb.z2[100000:140000]/np.mean(bb.z2[:40000])*50-49, 'r--',label='z', alpha=(0.5))
ax.plot(range(40000), bb.Iz[100000:140000]/np.mean(bb.Iz[:40000])*50-49,label='Iz', alpha=(0.5))
ax.legend()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [58]:
_,ax = plt.subplots(figsize=(9.2,4))
# ax.plot(range(40000), x_sync[:40000]/np.mean(x_sync[:40000]), label='z_fb')
ax.plot(range(40000), bb.z2[100000:140000]/np.mean(bb.z2[:40000]), 'r--',label='Iz', alpha=(0.5))
ax.legend()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### Scan Y

In [16]:
i = 10
fname = r'/data/new_trap/20200924/Bead1/Shaking/Shaking345/NoSpinShaking0_'+str(i)+'.h5'
bb = BeadDataFile.BeadDataFile(fname=fname)
bb.bead_height, np.mean(bb.cant_pos[1])*50

(176.17207316774534, 207.86780005340114)

In [11]:
385.2-np.mean(bb.cant_pos[0])*50-7.6/2+1

39.91469808346223

In [28]:
431.25-25*9.5-np.mean(bb.cant_pos[1])*50, 431.25-25*8

(-14.11780005340114, 231.25)

In [8]:
def getY_scanY(i):
    print(i,'  ******************')
    fname = r'/data/new_trap/20200924/Bead1/Shaking/PatchCharges_345/SpinShaking0_'+str(i)+'.h5'
    bb = BeadDataFile.BeadDataFile(fname=fname)
    return np.mean(bb.cant_pos[1])*50

In [30]:
def getAmp_scanY(i):
    print(i,'  ******************')
#     fname = r'/data/new_trap/20200924/Bead1/Shaking/PatchCharges_345/SpinShaking0_'+str(i)+'.h5'
    fname = r'/data/new_trap/20200924/Bead1/Shaking/Shaking345/SpinShaking0_'+str(i)+'.h5'

    bb = BeadDataFile.BeadDataFile(fname=fname)
    freq_, fft_ = bb.psd2('x')
    return [fft_[freq==harm*3] for harm in range(1,14)]

In [418]:
def getHeight_scanY(i):
    print(i,'  ******************')
    fname = r'/data/new_trap/20200924/Bead1/Shaking/PatchCharges_345/SpinShaking0_'+str(i)+'.h5'
#     fname = r'/data/new_trap/20200924/Bead1/Shaking/Shaking345/SpinShaking0_'+str(i)+'.h5'
#     fname = r'/data/new_trap/20200924/Bead1/Shaking/Shaking345/SpinNoShaking0_'+str(i)+'.h5'

    bb = BeadDataFile.BeadDataFile(fname=fname)
    return np.mean(bb.y2)

In [307]:
getHeight_scanY(500)

500   ******************


180.36127286527625

In [21]:
def getPhase_scanY(i):
    print(i,'  ******************')
#     fname = r'/data/new_trap/20200924/Bead1/Shaking/PatchCharges_345/NoSpinShaking0_'+str(i)+'.h5'
    fname = r'/data/new_trap/20200924/Bead1/Shaking/Shaking345/SpinShaking0_'+str(i)+'.h5'
    bb = BeadDataFile.BeadDataFile(fname=fname)
#     freq_, fft_ = bb.psd2('x')
    x = bb.x2
    fft_ = np.fft.rfft(x,norm='ortho')
    freq_ = np.fft.rfftfreq(len(x), d=1./5000)
    return [np.angle(fft_[freq==harm*3]) for harm in range(1,14)]

In [10]:

# test_y = Parallel(n_jobs=20)(delayed(getY_scanY)(i) for i in tqdm(range(4000)))

In [220]:
test_amps2 = np.array(Parallel(n_jobs=20)(delayed(getAmp_scanY)(i) for i in tqdm(range(3000)))).T[0]


100%|██████████| 3000/3000 [00:09<00:00, 285.30it/s]

In [205]:
test_height2 = test_height

In [419]:
test_y = Parallel(n_jobs=20)(delayed(getHeight_scanY)(i) for i in tqdm(range(3000)))




  0%|          | 0/3000 [00:00<?, ?it/s]


  3%|▎         | 80/3000 [00:00<00:06, 472.06it/s]


  4%|▍         | 120/3000 [00:00<00:07, 395.36it/s]


  8%|▊         | 240/3000 [00:00<00:06, 452.63it/s]


 11%|█         | 320/3000 [00:00<00:05, 479.20it/s]


 13%|█▎        | 400/3000 [00:00<00:05, 483.91it/s]


 16%|█▌        | 480/3000 [00:01<00:05, 439.61it/s]


 19%|█▊        | 560/3000 [00:01<00:05, 426.32it/s]


 21%|██▏       | 640/3000 [00:01<00:05, 415.80it/s]


 24%|██▍       | 720/3000 [00:01<00:05, 387.60it/s]


 27%|██▋       | 800/3000 [00:01<00:06, 358.36it/s]


 29%|██▉       | 880/3000 [00:02<00:05, 369.89it/s]


 32%|███▏      | 960/3000 [00:02<00:05, 365.40it/s]


 35%|███▍      | 1040/3000 [00:02<00:05, 374.85it/s]


 37%|███▋      | 1120/3000 [00:02<00:04, 378.16it/s]


 40%|████      | 1200/3000 [00:02<00:04, 386.30it/s]


 43%|████▎     | 1280/3000 [00:03<00:04, 395.52it/s]


 45%|████▌     | 1360/3000 [00:03<00:04, 384.83it/s]


 48%|████▊     | 1440/3000 [00:0

In [22]:
test_phase = np.array(Parallel(n_jobs=20)(delayed(getPhase_scanY)(i) for i in tqdm(range(1000)))).T[0]


100%|██████████| 1000/1000 [00:34<00:00, 29.62it/s]

In [146]:
np.mean(np.array(test2_9hz).reshape(-1,10), axis=1).shape

(400,)

In [78]:
import obspy
from obspy.signal.detrend import polynomial
data = np.array(test_Iz[:3000])
polynomial(data, order=3, plot=True)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

array([-31.84974983, -31.28241833, -44.64879813, ..., -66.38842589,
       -65.54251498, -63.30591768])

In [421]:
_,ax = plt.subplots(3,1,figsize=(9,9), sharex=True)
i = 3
# ax[0].scatter(test_y[:3000], np.sqrt(test_amps[i][:3000])*x_scale1, label=str(np.arange(1,13)[i]*3)+' y-scan')
# ax[0].scatter(test_y[:3000], np.sqrt(test_amps2[i])*x_scale1, label=str(np.arange(1,13)[i]*3)+' constant')
# ax[0].legend()

bla = np.arange(4000)
# bla = test_y
ax[0].scatter(bla[:3000], test_x[:3000], label='y-scan')
# ax[0].scatter(bla[:3000], test_phase0_2[:3000], label='y-fixed')
# ax[0].scatter(bla[:1000], test_Iz_noshaking[:1000], label='no-shaking')
ax[0].set(ylabel='carrier phase - one quadrant')

ax[1].scatter(bla[:3000], np.array(test_y[:3000]), label='y-scan')
# ax[1].scatter(bla[:3000], np.array(test_amp4_2[:3000])*-1, label='y-fixed')
# ax[1].scatter(bla[:1000], np.array(test_spin_noshaking[:1000])*-1, label='no-shaking')
ax[1].set(ylabel='Z-PD amplitude [a.u.]')

ax[2].scatter(bla[:3000], test_z[:3000], label='y-scan')
# ax[2].scatter(bla[:3000], test_z_2[:3000], label='y-fixed')
# ax[2].scatter(bla[:1000], test_quadsum_noshaking[:1000], label='no-shaking')
ax[2].set(ylabel='Z [a.u.]', xlabel='Y [$\mu$m]')

[a.legend() for a in ax]
# ax[3].scatter(test_y[:3000], test_phase0[:3000])
# ax[3].scatter(test_y[:1000], test_phase0_noshaking[:1000])
# ax[0].scatter(test_y[:3000], test_height[:3000],label='y-scan shaking')
# ax[0].scatter(test_y[:3000], test_height2[:3000],label='y-fixed shaking')
# ax[0].scatter(test_y[:1000], test_height_noshaking[:1000],label='no shaking')


# ax[1].scatter(test_y[:3000], test_Iz2[:3000])


# ax[3].set(xlabel='Y [$\mu$m]', ylabel='Power (from spin) [a.u.]')
# ax[2].scatter(test_y[:3000], test_quadsum2[:3000])
# ax.semilogy(np.mean(np.array(test1_zfb).reshape(-1,5), axis=1), np.mean(np.sqrt(test2_9hz).reshape(-1,5), axis=1)*x_scale1, '.')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [80]:
_,ax = plt.subplots(figsize=(9,4))
i = 5
ax.semilogy(test_y[:3000], np.sqrt(test_amps[i][:3000])*x_scale1, label=str(np.arange(1,13)[i]*3)+' y-scan')
ax.semilogy(test_y[:3000], np.sqrt(test_amps2[i])*x_scale1, label=str(np.arange(1,13)[i]*3)+' constant')
ax.legend()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [18]:
test_phase2 = np.array(test_phase2).T[0]

In [23]:
_,ax = plt.subplots(2,1,figsize=(9,6), sharex=True)
i = 7
ax[0].scatter(test_y[:3000], test_phase[i][:3000], label=str(np.arange(1,13)[i]*3)+' spinning')
ax[0].scatter(test_y[:3000], test_phase2[i][:3000], label=str(np.arange(1,13)[i]*3)+' not spinning', alpha=0.6)
ax[0].legend()
ax[0].set(ylabel='Phase [rad]')
# ax[1].scatter(test_y[:3000], np.sqrt(test_amps[i][:3000])*x_scale1, label=str(np.arange(1,13)[i]*3)+' y-scan')
# ax[1].scatter(test_y[:3000], np.sqrt(test_amps2[i])*x_scale1, label=str(np.arange(1,13)[i]*3)+' y-fixed', alpha=0.6)
ax[1].legend()
ax[1].set(ylabel='Amplitude [N/$\sqrt{Hz.}$]', xlabel='Y [$\mu$m]')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

NameError: name 'test_y' is not defined

In [24]:
_,ax = plt.subplots(3,4,figsize=(9,8.5), sharex=True)
# i = 0
_ = [a.hist(test_phase[j][:3000]%(2*np.pi), label=str(np.arange(1,13)[j]*3)+'Hz. spin', bins=60, range=(0,2*np.pi)) for j,a in enumerate(ax.flatten())]
_ = [a.hist(test_phase2[j][:3000]%(2*np.pi), label=str(np.arange(1,13)[j]*3)+'Hz. no-spin', alpha=0.5, bins=60, range=(0,2*np.pi)) for j,a in enumerate(ax.flatten())]
# _ = ax.hist(test_phase2[i][:3000]%(2*np.pi), label=str(np.arange(1,13)[i]*3)+' constant', alpha=0.6, bins=60, range=(0,2*np.pi))
_ = [a.legend() for a in ax.flatten()]
_ = [a.set(xlabel='phase [rad]') for a in ax.flatten()]

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### Look at the spin

In [394]:
bb.spin_data[0].shape

(500000,)

In [39]:
datafile_num = 2500
datafile_num1 = 2600
fname = r'/data/new_trap/20200924/Bead1/Shaking/PatchCharges_345/SpinShaking0_'+str(datafile_num)+'.h5'
bb = BeadDataFile.BeadDataFile(fname=fname)
fname = r'/data/new_trap/20200924/Bead1/Shaking/PatchCharges_345/SpinShaking0_'+str(datafile_num1)+'.h5'
bb1 = BeadDataFile.BeadDataFile(fname=fname)
_,ax = plt.subplots()
aaa2, bbb2 = matplotlib.mlab.psd(bb.spin_data[0], Fs = 50000, NFFT = 500000)
ax.semilogy(bbb2, aaa2, label='file #: '+str(datafile_num))
aaa1, bbb1 = matplotlib.mlab.psd(bb1.spin_data[0], Fs = 50000, NFFT = 500000)
ax.semilogy(bbb1, aaa1, label='file #: '+str(datafile_num1), alpha=0.5)
ax.legend()
ax.set(xlim=(17450,17550), xlabel='Frequency [Hz.]', ylabel='spin-psd[a.u.]')
# print(np.mean(aa[(bb>60)*(bb<82)])*1e10)
# cutoff = int(30000/10)
# bb[np.argmax(aa[cutoff:])+cutoff]

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

[(17450.0, 17550.0),
 Text(0.5, 0, 'Frequency [Hz.]'),
 Text(0, 0.5, 'spin-psd[a.u.]')]

In [14]:
## likelihood calculator stuff
ll = likelihood_analyser.LikelihoodAnalyser()
gravity_freqs = np.array([36, 33, 42, 18, 21, 39, 24])

In [23]:
bdf_z = bdf_discharge[40:45]
fname = r'/data/new_trap/20200320/Bead1/TransFunc/Repeat_TransFunc_20200104/TransFunc_X_m300k_250s_1hz.h5'
tf_x = BeadDataFile.BeadDataFile(fname)
fname = r'/data/new_trap/20200320/Bead1/TransFunc/Repeat_TransFunc_20200104/TransFunc_Y_m300k_250s_1hz.h5'
tf_y = BeadDataFile.BeadDataFile(fname)
fname = r'/data/new_trap/20200320/Bead1/TransFunc/Repeat_TransFunc_20200104/TransFunc_Z_m300k_250s_1hz.h5'
tf_z = BeadDataFile.BeadDataFile(fname)
gfw.build_transfer_function([tf_x, tf_y, tf_z], base_freq=1, number_of_harmonics=10, scale_freq=71, plot=False)

No height data
No cantilever data
Error loading spinning data
No laser power data
No height data
No cantilever data
Error loading spinning data
No laser power data
No height data
No cantilever data
Error loading spinning data
No laser power data


In [30]:
bandwidth = 2
m1_zscale = gfw.build_z_response(bdf_z, drive_freq=71, charges=3, bandwidth=bandwidth, decimate=10)
gravity_scales = np.array([gfw.scale_Z2])* np.interp(gravity_freqs, gfw.tf_freq, gfw.tf_ffts[2])
noises_std = np.ones(len(gravity_freqs))

***************************************************
Z2-amplitude:  2.42e+02
reduced chi2:  5476.558157447907
***************************************************
Z2-amplitude:  2.39e+02
reduced chi2:  3034.748270618688
***************************************************
Z2-amplitude:  2.50e+02
reduced chi2:  6858.95396814926
***************************************************
Z2-amplitude:  2.59e+02
reduced chi2:  5415.162615180254
***************************************************
Z2-amplitude:  2.39e+02
reduced chi2:  2546.787668771445
Z2 response (amplitude): 245.73717124858416


In [47]:
def func2(i):
    print(i,'  ******************')
    fname = r'/data/new_trap/20200924/Bead1/Shaking/Shaking345/SpinShaking0_'+str(i)+'.h5'
    bb = BeadDataFile.BeadDataFile(fname=fname)

    ## force template
    dist = 40
    z_sep = -20
    y_offset = 0
    test_shaking378 = force_vs_time(dist*1e-6,z_sep*1e-6,200e-6,3,"z",1000e-6, yuk_or_grav="yuk",offset_y=y_offset*1e-6, 
                     alpha=1e8, bead_size=3.8e-6)
    shaking378_template = np.array(test_shaking378[1])*1.85/1.55

    fit_kwargs = {'A': 0, 'phi': 0.2, 'sigma':10.7,
              'error_A': 1, 'error_phi': 0.5, 'errordef': 1, 'error_sigma': 10,
              'limit_phi': [0, 2 * np.pi],
              'limit_A': [-1000, 1000], 'limit_sigma': [0, 1000000],
              'print_level': 0, 'fix_A': False, 'fix_phi': True, 'fix_sigma': False}
    mm_tmp = ll.find_mle_multiHarmoincs(x=bb.z2, template=shaking378_template, scales=gravity_scales, 
                                          signal_freqs=gravity_freqs, bandwidth=bandwidth, noises=noises_std,
                                          decimate=10, **fit_kwargs)

    return mm_tmp.values[0]

In [48]:
test_mle = np.array(Parallel(n_jobs=40)(delayed(func2)(i) for i in tqdm(range(1000))))

100%|██████████| 1000/1000 [00:16<00:00, 56.97it/s]


In [49]:
np.mean(test_mle), np.std(test_mle), np.std(test_mle)/np.sqrt(1000)

(-64.307978991231, 308.21590678043725, 9.746642765203164)

In [46]:
_,ax = plt.subplots()
data = list(test_mle)#+list(mm_mle_A10)
avg = np.mean(data)
var = np.var(data)
# From that, we know the shape of the fitted Gaussian.
pdf_x = np.linspace(-25,25,100)
pdf_y = 1.0/np.sqrt(2*np.pi*var)*np.exp(-0.5*(pdf_x-avg)**2/var)

ax.hist(data, range=(-25,25), bins=25, density=True)
# ax.hist(mm_mle_A10, range=(-25,25), bins=30, density=True, alpha=0.4)
# Then we plot :
ax.set(xlim=(-25,25), xlabel=r'$\hat{\alpha} \,[10^{8}]$')
ax.plot(pdf_x,pdf_y,'k--')

print('alpha MLE: ', avg, 'std: ', np.std(data)/np.sqrt(1000))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

alpha MLE:  -0.6425047548183118 std:  0.09407395427994644
